In [1]:
import os, warlock, json
from datetime import datetime
import time
from jsonschema import validate
import dissinetpytools.dator as dator
from dotenv import load_dotenv

import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.filterwarnings("ignore")

import numpy as np
from copy import deepcopy
from shutil import copyfile

import uuid
import dill as pickle
from DataHolder import DataHolder
import inkimplib


# type hinting
from collections.abc import Sequence, Callable
from typing import List, Dict, Tuple


Creating the DataHolder singleton object


In [2]:
import json
import os

file_path = "../datasets/reimport/entities.json"

# Check if file exists and is not empty
if not os.path.isfile(file_path):
    print("File does not exist.")
elif os.stat(file_path).st_size == 0:
    print("File is empty.")
else:
    # Validate JSON content
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        print("JSON is valid")
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


JSON is valid


In [47]:
# read latest data real entities
ld_df = pd.read_json('./data/latest/entities.json').fillna("")
ld_dfr = ld_df.copy()

# read new entities
ri_df = pd.read_json("../datasets/reimport/entities.json").fillna("")
ri_dfr = ri_df.copy()

In [6]:
ld_df[ld_df['id']=="T0"]

,class,data,detail,id,isTemplate,label,language,legacyId,notes,props,references,status,createdAt,updatedAt,elementsOrders,right,usedTemplate,entities,warnings
22564,T,"{'parent': False, 'protocol': {'description': ...",,T0,NaN,root,eng,None,[],[],None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
ld_df[ld_df['id']=="b6b9e80b-4769-4827-bedb-179433836250"]['references'].values # b6b9e80b-4769-4827-bedb-179433836250

array([list([{'id': '693cec10-c498-4ca4-a1a6-833b26e62f44', 'resource': 'a824da24-6d0a-4fb4-adc4-103a9cc66b1c', 'value': 'c4eab0ab-81d3-4c31-892b-94cd3e6e8c96'}])],
      dtype=object)

In [13]:
ld_df.loc[ld_df['id']=="b6b9e80b-4769-4827-bedb-179433836250","label"] = 'Berardus de Camerino Modified'

In [14]:
ld_df[ld_df['id']=="b6b9e80b-4769-4827-bedb-179433836250"]

,class,data,detail,id,isTemplate,label,language,legacyId,notes,props,references,status,createdAt,updatedAt,elementsOrders,right,usedTemplate,entities,warnings
25758,P,{'logicalType': '1'},,b6b9e80b-4769-4827-bedb-179433836250,NaN,Berardus de Camerino Modified,lat,P0301_R0030,[Import batch [January2023] 2023-01-21 18:58:2...,"[{'bundleEnd': False, 'bundleOperator': 'a', '...",[{'id': '693cec10-c498-4ca4-a1a6-833b26e62f44'...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#ld_df.to_json('../../../../inkvirim/entities.json', orient='records')

In [48]:
import json

def find_all_values(json_obj, key="value"):
    """
    Recursively parse the JSON object to find all values of the specified key.

    :param json_obj: The JSON object (can be a dict or a list).
    :param key: The key to search for in the JSON object (default is "value").
    :return: A list of values associated with the specified key.
    """
    values = []

    if isinstance(json_obj, dict):
        for k, v in json_obj.items():
            if k == key:
                if "entityId" in v:
                  values.append(v['entityId'])
                else:
                  values.append(v)
            if isinstance(v, (dict, list)):
                values.extend(find_all_values(v, key))
    elif isinstance(json_obj, list):
        for item in json_obj:
            values.extend(find_all_values(item, key))

    return values


ld_dfr['value_objects_props'] = ld_dfr.apply(lambda x: find_all_values(x['props'], key="value") if "_R0030" in x['legacyId'] else [], axis=1)
ld_dfr['value_objects_references'] = ld_dfr.apply(lambda x: find_all_values(x['references'], key="value") if "_R0030" in x['legacyId'] else [], axis=1)
ld_dfr['value_objects_data'] = ld_dfr.apply(lambda x: find_all_values(x['data'], key="value") if "_R0030" in x['legacyId'] else [], axis=1)

ri_dfr['value_objects_props'] = ri_dfr.apply(lambda x: find_all_values(x['props'], key="value") if "_R0030" in x['legacyId'] else [], axis=1)
ri_dfr['value_objects_references'] = ri_dfr.apply(lambda x: find_all_values(x['references'] if "_R0030" in x['legacyId'] else [], key="value"), axis=1)
ri_dfr['value_objects_data'] = ri_dfr.apply(lambda x: find_all_values(x['data'] if "_R0030" in x['legacyId'] else [], key="value"), axis=1)

In [50]:
import itertools

old_value_objects = ld_dfr['value_objects_props'].to_list() +  ld_dfr['value_objects_references'].to_list() +ld_dfr['value_objects_data'].to_list()

new_value_objects = ri_dfr['value_objects_props'].to_list() +  ri_dfr['value_objects_references'].to_list() +ri_dfr['value_objects_data'].to_list()

old_value_objects = list(itertools.chain(*old_value_objects))
new_value_objects = list(itertools.chain(*new_value_objects))

In [41]:
len(old_value_objects), len(new_value_objects)

(8875, 9989)

In [51]:
# persons
# simple, no relation, no special related object around
# persons are just main entity plus props

# remove bologna persons from latest
ld_df_nobolognap = ld_df[~ld_df['legacyId'].str.contains("_R0030")].copy()
ld_df_nobolognap = ld_df_nobolognap[~(ld_df_nobolognap['id'].isin(old_value_objects) & (ld_df_nobolognap['class']=="V"))] # remove old value objects

# limit new parse just to bologna persons
ri_df_persons = ri_df[ri_df['legacyId'].str.contains("_R0030")].copy()  
ri_df_persons_vo = ri_df[ri_df['id'].isin(new_value_objects) & (ri_df['class']=="V")].copy() # add value objects  


result_df = pd.concat([ld_df_nobolognap,ri_df_persons,ri_df_persons_vo])

In [52]:
ri_df_persons_vo

,id,class,data,label,detail,language,notes,props,status,references,legacyId
2303,8b72a004-455f-4072-8ffb-54e9d0b8ac17,V,{'logicalType': '1'},Albertus,,,[Import batch [ReImport 06/2024] 2024-06-01 20...,[],1.0,[],
2304,ef831331-5860-4d3d-8893-3daf86944539,V,{'logicalType': '1'},de Bayso,,,[Import batch [ReImport 06/2024] 2024-06-01 20...,[],1.0,[],
2305,28dd880c-caef-4f86-8c87-90b140e5fd99,V,{'logicalType': '1'},"107, 108 (74)",,,[Import batch [ReImport 06/2024] 2024-06-01 20...,[],1.0,[],
2306,efb71cc1-5a86-42c4-b803-405c750c495a,V,{'logicalType': '1'},Andreas,,,[Import batch [ReImport 06/2024] 2024-06-01 20...,[],1.0,[],
2307,6bc437c2-8558-46de-b05b-249f62bd228f,V,{'logicalType': '1'},605 (817),,,[Import batch [ReImport 06/2024] 2024-06-01 20...,[],1.0,[],
...,...,...,...,...,...,...,...,...,...,...,...
8267,6421687c-f7db-471a-a104-f700a538c7af,V,{'logicalType': '1'},446 (661),,,[Import batch [ReImport 06/2024] 2024-06-01 20...,[],1.0,[],
8268,31bec75e-104d-48ef-9972-bcabaf2a8a59,V,{'logicalType': '1'},161 (144),,,[Import batch [ReImport 06/2024] 2024-06-01 20...,[],1.0,[],
8269,5e32cda2-9414-4c5e-b34c-a5784144178f,V,{'logicalType': '1'},3,,,[Import batch [ReImport 06/2024] 2024-06-01 20...,[],1.0,[],
8270,28dad185-6b21-4d6b-ac2c-4e765fce7a68,V,{'logicalType': '1'},8,,,[Import batch [ReImport 06/2024] 2024-06-01 20...,[],1.0,[],


In [55]:
result_df[result_df['id'].str.contains("3bf5842d-5742-474f-9bf7-0e56485c8f42")]

,class,data,detail,id,isTemplate,label,language,legacyId,notes,props,references,status,createdAt,updatedAt,elementsOrders,right,usedTemplate,entities,warnings
8265,V,{'logicalType': '1'},,3bf5842d-5742-474f-9bf7-0e56485c8f42,NaN,468,,,[Import batch [ReImport 06/2024] 2024-06-01 20...,[],[],1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
result_df.to_json('../../../../inkvirim/entities.json', orient='records')

In [26]:
# audits
lda_df = pd.read_json('./data/latest/audits.json').fillna("")
lda_dfr = lda_df.copy()

# read new entities
ria_df = pd.read_json("../datasets/reimport/audits.json").fillna("")
ria_dfr = ria_df.copy()


In [29]:
ria_dfr = ria_df.copy()

In [10]:
len(lda_df), len(ria_df)

(102066, 16510)

In [33]:
ria_dfr['about_bologna_persons'] = ria_dfr.apply(lambda x: True if 'legacyId' in x['changes'] and "_R0030" in x['changes']['legacyId'] else False  ,axis=1)

# transform time
from datetime import datetime, timezone

def to_reql_time(date_str):
    # print("*",str(date_str))
    
    if str(date_str)[0] == "2":
      # Parse the date string to a datetime object
      
      dt = datetime.fromisoformat(str(date_str))
  
      # Convert datetime to epoch time
      epoch_time = int(dt.timestamp())
  
      # Extract timezone
      timezone_str = date_str[-6:]
  
      # Create the dictionary
      reql_time = {
          '$reql_type$': 'TIME',
          'epoch_time': epoch_time,
          'timezone': timezone_str
      }
      return reql_time
    else:
      return date_str

ria_dfr['date'] = ria_dfr['date'].astype(str)
ria_dfr['date'] = ria_dfr['date'].apply(lambda x: to_reql_time(x))

* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:02+00:00
* 2024-06-01 20:18:03+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:04+00:00
* 2024-06-01 20:18:0

In [34]:
audits_df = pd.concat([lda_df,ria_dfr[ria_dfr['about_bologna_persons']].drop(columns=["about_bologna_persons"])])

In [35]:
audits_df[audits_df['changes'].apply(lambda x: True if 'id' in x and x['id'] == "eba0c4e7-db9a-4478-b4f5-cc071732214c" else False)]

,changes,date,entityId,id,user
59503,"{'class': 'P', 'data': {'logicalType': '1'}, '...","{'$reql_type$': 'TIME', 'epoch_time': 16743282...",eba0c4e7-db9a-4478-b4f5-cc071732214c,950bc259-ee55-4731-b621-d69ce1ca33c4,103
11224,"{'class': 'P', 'id': 'eba0c4e7-db9a-4478-b4f5-...","{'$reql_type$': 'TIME', 'epoch_time': 17172733...",eba0c4e7-db9a-4478-b4f5-cc071732214c,935d80f1-192d-4692-b770-d30b3f32c50a,103


In [36]:
audits_df.to_json('../../../../inkvirim/audits.json', orient='records')

In [37]:
import zipfile

json_string = audits_df.to_json(orient='records')

with zipfile.ZipFile('../../../../inkvirim/audits.zip', "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write("../../../../inkvirim/audits.json")
